# Google Map
- 換ip:proxy list,safeip
- 換headers
- 多線進程:grequest, greenlet(可用手機換ip)
- 步驟:1.selenium自動景點搜尋
- ........2.法一:原圖 w/ XML,法二:縮圖 w/ HTML上的links 
- <img src="mapImg.png">


## GoogleMap Img w/ 原圖(單點)
- 按下"201張照片", JS生成一.json檔含100張圖片原始與縮圖連結.
- 手動輸入搜尋地點, 開啟Neteork下的XHR:photo?author=0..., 送出RequestURL接收Response:.json
- 目前有驗證的問題:也許可使用Ajax解決

In [ ]:
#完整版

import requests
from bs4 import BeautifulSoup as bs
import io
import json
import re
import os

loc = "Awaji Island"
img_ReqUrl={}
reqUrl='https://www.google.com.tw/maps/preview/photo?authuser=0&hl=zh-TW&pb=!1e2!3m2!1s0x3554ba6dbfc4cfe1%3A0xaad592bb86ce307e\
!9e0!5m42!2m2!1i203!2i100!3m1!2i100!7m35!1m3!1e1!2b0!3e3!1m3!1e2!2b1!3e2!1m3!1e2!2b0!3e3!1m3!1e3!2b0!3e3!1m3!1e4!2b0!3e3!1m3!1e8!2b0!3e3\
!1m3!1e3!2b1!3e2!1m4!1e5!2b0!3e1!4e2!2b1!4b1!9b0!6m3!1sjhJlV6jMHsGZ0gTz47HQDA!7e81!15i16698'

headers = {'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:46.0) Gecko/20100101 Firefox/46.0'} 
res = requests.get(reqUrl, headers = headers)

#文字轉.json
imgjson = res.text.split(')]}\'')[1]
with io.open('E:/ProjectTravel/IMG/temp.json', 'w', encoding='utf-8') as file:
    file.write(imgjson)
    
#.json讀入
with open('E:/ProjectTravel/IMG/temp.json', 'r') as f:
    hrefs = json.load(f)

#改寫連結
imghrefs=[]
for i in xrange(0,len(hrefs[0])):
    #print hrefs[0][i][0]
    #print "-------------------------------------------------------------------------"
    if re.search('^(\d)+$',hrefs[0][i][0]):
        imghrefs.append("https://static.panoramio.com.storage.googleapis.com/photos/1920x1280/"+hrefs[0][i][0]+".jpg") 
        print imghrefs[-1]
    elif re.search('^-(.{22,})',hrefs[0][i][0]):
        imghrefs.append("https://lh3.ggpht.com/" + hrefs[0][i][0] + "/s2000/photo") 
        print imghrefs[-1]
    elif re.search('^http',hrefs[0][i][0]):
        imghrefs.append(hrefs[0][i][0])
        print imghrefs[-1]
    else:
        print hrefs[0][i][0]

#連結寫出保存     
img_ReqUrl[loc]=[reqUrl,imghrefs]
with open('E:/ProjectTravel/IMG/img_ReqUrl.json' , 'a') as f:
    json.dump(img_ReqUrl, f)
          
#動態建立資料夾
dir = loc
os.mkdir('E:/ProjectTravel/IMG/'+dir)

#下載
import requests
import shutil 
for i in xrange(0,len(imghrefs)):
    url = imghrefs[i]
    res = requests.get(url, stream=True)
    fname = loc+'{:0>4}'.format(i+1)+'.jpg'
    with open('E:/ProjectTravel/IMG/Awaji Island/'+fname, 'wb') as f:
        shutil.copyfileobj(res.raw, f)

In [ ]:
# 連結寫出保存     
img_ReqUrl[loc]=[reqUrl,imghrefs]
with open('E:/ProjectTravel/IMG/img_ReqUrl.json' , 'a') as f:
    json.dump(img_ReqUrl, f)
        
#動態建立資料夾
dir = loc
os.mkdir('E:/ProjectTravel/IMG/'+dir)

#下載
import requests
import shutil 
for i in xrange(0,len(imghrefs)):
    url = imghrefs[i]
    res = requests.get(url, stream=True)
    fname = loc+'{:0>4}'.format(i+1)+'.jpg'
    with open('E:/ProjectTravel/IMG/'+dir+'/'+fname, 'wb') as f:
        shutil.copyfileobj(res.raw, f)

## GoogleMap Img w/ 縮圖(多點)
- 自動輸入地點,改寫縮圖連結,少於100張圖片
- continue,break,pass：https://docs.python.org/2/tutorial/controlflow.html

In [64]:
#使用googlemap搜尋景點,取得景點相片連結

import codecs,os
with codecs.open('C:/Users/BIG DATA/Downloads/jptoursmall..txt', 'r', encoding='utf-8') as file:
    attr = file.readlines()
uniattr=[attr[i].strip() for i in xrange(0,len(attr))]

for i in xrange(0,len(uniattr)):
    loc=uniattr[i]
    if os.path.isdir('E:/ProjectTravel/IMG/'+loc):
        #print "yes"+loc
        continue
    else:    
        from selenium import webdriver
        from selenium.webdriver.common.by import By
        from selenium.webdriver.common.keys import Keys
        from selenium.webdriver.support.ui import Select
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC
        from selenium.common.exceptions import NoSuchElementException
        from selenium.common.exceptions import NoAlertPresentException
        from selenium.webdriver.common.action_chains import ActionChains
        import unittest, time, re
        from bs4 import BeautifulSoup 
        import json
        import codecs

        driver = webdriver.Firefox()
        #放到最大避免找不到element
        driver.set_window_size(1024, 600)
        driver.maximize_window()
        driver.implicitly_wait(30) #最多等30s
        base_url = "https://www.google.com.tw"
        driver.get(base_url + "/maps")
        #wait直到JS生成搜尋鈕:
        wait = WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.scene-footer-container.noprint')))
        driver.find_element_by_id("searchboxinput").send_keys(loc)
        #移動到搜尋鈕並按下
        search = driver.find_element_by_css_selector("button.searchbox-searchbutton") 
        ActionChains(driver).move_to_element(search).click(on_element=None).perform()

        try:
            if len(driver.find_elements_by_css_selector("label.widget-pane-section-image-pack-label")) < 2:
                print uniattr[i]
                driver.quit()
                continue
        except:
            print uniattr[i]
            driver.quit()

        else:
            #取得google景點名
            soup = BeautifulSoup(driver.page_source,'html.parser')
            wait = WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.widget-pane-section-header-title')))
            engloc = soup.select("div.widget-pane-section-header-title h1")[0].text
            driver.find_elements_by_css_selector("button.widget-pane-section-header-hero.widget-pane-fading.widget-pane-fade-in.widget-pane-section-header-hero-clickable")[-1].click()          
            #"XX 張相片"
            qty=[int(howmany.text.split()[0]) for howmany in soup.select('label.widget-pane-section-image-pack-label > span') if re.search('^\d{1,3}', howmany.text)]                
            #"相片"
            try:
                if qty[0] > 100 :
                    qty[0] = 100
                elif qty[0] < 10 :
                    qty[0] = 1
                elif len(qty) == 0:
                    print "unknow howmany"+uniattr[i]
            except:
                qty.append(1)
                print "UNKNOWN"+uniattr[i]
            
            #等待取得小圖連結
            soup = BeautifulSoup(driver.page_source,'html.parser')
            while len(soup.select('img.widget-runway-card-background-flicker-hack')) <= qty[0]*0.8:
                print len(soup.select('img.widget-runway-card-background-flicker-hack')),qty[0]
                lastImg=driver.find_element_by_css_selector("button.widget-runway-photo-upload-button")
                lastImg.location_once_scrolled_into_view
                search = driver.find_elements_by_css_selector("button.widget-runway-card-button")[-1]
                ActionChains(driver).move_to_element(search).click(on_element=None).perform()
                time.sleep(30)
                soup = BeautifulSoup(driver.page_source,'html.parser')
            #改寫連結
            locImgHref=[]
            for l in soup.select('img.widget-runway-card-background-flicker-hack'):
                if "proxy" in l["src"]:
                    locImgHref.append("https:" + l["src"].split("=w")[0] + "=w5000") 
                    #print locImgHref
                elif "/w" in l["src"]:
                    locImgHref.append("https:" + l["src"].split("/w")[0] + "/w5000/photo") #/w:可能環景圖
                    #print l["src"]
                elif "ggpht.com" in l["src"]:
                    continue
                else:
                    locImgHref.append("https:" + l["src"].split("/s")[0] + "/s5000/photo")
                    #print l["src"]
            driver.quit()

            #連結寫出保存
            imgsmall_ReqUrl={}
            imgsmall_ReqUrl[loc]=[engloc,locImgHref]
            with open('E:/ProjectTravel/IMG/imgSmall_ReqUrl.json', 'a') as f:
                json.dump(imgsmall_ReqUrl, f)           
            #動態建立資料夾
            import os
            loc=uniattr[i]
            os.mkdir('E:/ProjectTravel/IMG/'+loc)
            #下載
            import requests
            import shutil 
            for i in xrange(0,len(locImgHref)):
                url = locImgHref[i]
                res = requests.get(url, stream=True)
                fname = loc+'{:0>4}'.format(i+1)+'.jpg'
                with open('E:/ProjectTravel/IMG/'+loc+'/'+fname, 'wb') as f:
                    shutil.copyfileobj(res.raw, f)

UNKNOWN﻿有樂町
0 1
0 44
0 97
0 53
東武百貨店池袋店
Terrace City
Coppice
台場Mediage
0 100
0 100
TOKYO CRUISE
0 15
江戶城
アカチャンホンポ アルカキット錦糸町店
忠犬八公
友都八喜
0 31
0 80
慶應義塾大學
0 1
Tokyo Odaiba Oedo Onsen Monogatari


### GoogleMap w/ 原圖(多點)
- 進入照片特輯, 分析html, 找出特定字串
- 取得原始照片連結.json, 整理下載
- Element is not clickable:https://www.seleniumeasy.com/selenium-tutorials/element-is-not-clickable-at-point-selenium-webdriver-exception

In [ ]:
#自動載圖(被擋驗證不過)

import codecs
with codecs.open('C:/Users/BIG DATA/Downloads/jptour.txt', 'r', encoding='utf-8') as file:
    attr = file.readlines()
uniattr=[]
for i in xrange(0,len(attr)):
    uattr = attr[i].strip()
    uniattr.append(uattr)

for i in xrange(0,len(uniattr)):
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.support.ui import Select
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import NoSuchElementException
    from selenium.common.exceptions import NoAlertPresentException
    from selenium.webdriver.common.action_chains import ActionChains
    import unittest, time, re
    from bs4 import BeautifulSoup 
    import json,io,re,os
    import requests
    import shutil

    driver = webdriver.Firefox()
    #放到最大避免找不到element
    driver.set_window_size(1024, 600)
    driver.maximize_window()
    driver.implicitly_wait(30) #最多等30s
    driver.get("https://www.google.com.tw/maps")
    loc = uniattr[i]
    #wait直到JS生成搜尋鈕:
    wait = WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.scene-footer-container.noprint')))
    driver.find_element_by_id("searchboxinput").send_keys(loc)
    #移動到搜尋鈕並按下
    search = driver.find_element_by_css_selector("button.searchbox-searchbutton") 
    ActionChains(driver).move_to_element(search).click(on_element=None).perform()
    
    #照片
    waitpic = WebDriverWait(driver,30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button.widget-pane-section-image-pack-button')))
    searchpic = driver.find_elements_by_css_selector("button.widget-pane-section-image-pack-button")[0]
    ActionChains(driver).move_to_element(searchpic).click(on_element=None).perform()
    searchpic.click();
    WebDriverWait(driver, 30).until(lambda driver: driver.find_element_by_css_selector('.widget-pane-term-link'))
    WebDriverWait(driver, 30).until(lambda driver: driver.find_element_by_css_selector('a.fineprint-item.fineprint-padded.noprint'))
    time.sleep(30)
    html = driver.page_source
    soup = BeautifulSoup(str(html.encode('utf-8')),'html.parser')
    engloc = soup.select("div.widget-pane-section-header-title h1")[0].text #存英文景點名
    driver.quit()
    
    t1 = soup.select('a.gb_Me.gb_Ha.gb_rb')[0]['href']
    t1_match =  re.search('0x(.{37})%',t1).group(0)
    text1 = re.sub(":","%3A",t1_match)
    text2 = soup.select('.widget-pane-section-image-pack-button')[0]['jstrack']
    request_url = "https://www.google.com.tw/maps/preview/photo?authuser=0&hl=zh-TW&pb=!1e2!3m2!1s%s!9e0!5m42!2m2!1i203!2i100!3m1!2i100!7m35!1m3!1e1!2b0!3e3!1m3!1e2!2b1!3e2!1m3!1e2!2b0!3e3!1m3!1e3!2b0!3e3!1m3!1e4!2b0!3e3!1m3!1e8!2b0!3e3!1m3!1e3!2b1!3e2!1m4!1e5!2b0!3e1!4e2!2b1!4b1!9b0!6m3!1s%s!7e81!15i16698"%(text1,text2)
    #print text1,text2
    #print request_url
    headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:46.0) Gecko/20100101 Firefox/46.0',
    'Cookie':"NID=79=CPQLMgjGLkAavsAHRIAgZEuYDTfOo0fkHu_Oni9keVFAOoTbh0khCBg43390Rx-Qd8Vhakd11F0gCfKVJQC-v7ffiKjxGdXjf90px2Ly60ox-3Jfi-1Uaj1kYt6xZNDL7vvTZ7vl5Iy3dbw; GOOGLE_ABUSE_EXEMPTION=ID=b1c5d767ddcc97bd:TM=1466858065:C=c:IP=191.101.12.120-:S=APGng0uD_da7U7FrqGA5gRlIi0JVbB7wBQ; OGPC=5061821-1:; DV=svtzA9IFA1EVLEWZLaMIXfSZLBwPqwI"}
    res = requests.get(request_url,headers=headers)
    print res.text
    
    #文字轉.json
    imgjson = res.text.split(')]}\'')[1]
    with io.open('E:/ProjectTravel/IMG/temp.json', 'w', encoding='utf-8') as file:
        file.write(imgjson)    
    #.json讀入
    with open('E:/ProjectTravel/IMG/temp.json', 'r') as f:
        hrefs = json.load(f)
    #改寫連結
    imghrefs=[]
    for i in xrange(0,len(hrefs[0])):
        if re.search('^(\d)+$',hrefs[0][i][0]):
            imghrefs.append("https://static.panoramio.com.storage.googleapis.com/photos/1920x1280/"+hrefs[0][i][0]+".jpg") 
            #print imghrefs[-1]
        elif re.search('^-(.{23,})',hrefs[0][i][0]):
            imghrefs.append("https://lh3.ggpht.com/" + hrefs[0][i][0] + "/s2000/photo") 
            #print imghrefs[-1]
        elif re.search('^http',hrefs[0][i][0]):
            imghrefs.append(hrefs[0][i][0])
            #print imghrefs[-1]
        else:
            print hrefs[0][i][0]

    #連結寫出保存 
    img_ReqUrl={}
    img_ReqUrl[loc]=[engloc,imghrefs]
    with open('E:/ProjectTravel/IMG/img_ReqUrl.json' , 'a') as f:
        json.dump(img_ReqUrl, f)          
    #動態建立資料夾
    import os
    os.mkdir('E:/ProjectTravel/IMG/'+loc)
    #下載
    import requests
    import shutil 
    for i in xrange(0,len(imghrefs)):
        url = imghrefs[i]
        res = requests.get(url, stream=True)
        fname = loc+'{:0>4}'.format(i+1)+'.jpg'
        with open('E:/ProjectTravel/IMG/'+loc+'/'+fname, 'wb') as f:
            shutil.copyfileobj(res.raw, f)

In [106]:
#手動原圖(載XML)

import json,io,re,os
loc = u"仁和寺"
engloc = "Ninna-ji"
#.txt轉json
with io.open('C:/Users/BIG DATA/Downloads/f (31).txt', 'r', encoding='utf-8') as file:
    res = file.read()
imgjson = res.split(')]}\'')[1]
with io.open('E:/ProjectTravel/IMG/temp.json', 'w', encoding='utf-8') as file:
    file.write(imgjson)    
#.json讀入
with open('E:/ProjectTravel/IMG/temp.json', 'r') as f:
    hrefs = json.load(f)
#改寫連結
imghrefs=[]
for i in xrange(0,len(hrefs[0])):
    if re.search('^(\d)+$',hrefs[0][i][0]):
        imghrefs.append("https://static.panoramio.com.storage.googleapis.com/photos/1920x1280/"+hrefs[0][i][0]+".jpg") 
        #print imghrefs[-1]
    elif re.search('^-(.{23,})',hrefs[0][i][0]):
        imghrefs.append("https://lh3.ggpht.com/" + hrefs[0][i][0] + "/s2000/photo") 
        #print imghrefs[-1]
    elif re.search('^http',hrefs[0][i][0]):
        imghrefs.append(hrefs[0][i][0])
        #print imghrefs[-1]
    else:
        print hrefs[0][i][0]

#連結寫出保存 
img_ReqUrl={}
img_ReqUrl[loc]=[engloc,imghrefs]
with open('E:/ProjectTravel/IMG/img_ReqUrl.json' , 'a') as f:
    json.dump(img_ReqUrl, f)
        
#動態建立資料夾
import os
os.mkdir('E:/ProjectTravel/IMG/'+loc)
#下載
import grequests
import requests
import shutil 
for i in xrange(0,len(imghrefs)):
    url = imghrefs[i]
    res = requests.get(url, stream=True) #grequest:https://pypi.python.org/pypi/grequests
    fname = loc+'{:0>4}'.format(i+1)+'.jpg'
    with open('E:/ProjectTravel/IMG/'+loc+'/'+fname, 'wb') as f:
        shutil.copyfileobj(res.raw, f)

Xgh8clqZXuV_c6SC10pfrw
jzCz5scFfCF2zU6AxSkP3w
xxf3buZVrAOICcI-p2P1ew
0rY6xrdS4O5M0ZP5YlzBBQ
HOrZaZ3NAICLZq1NHknwbg
2bQybTl8pHA1WXeD5Ix7yg
sHuuHYBkTrQoE89ik0lYwA
jIr-33I8m4LvUzrF87QzAQ
NNL7RpMRUkmsCQKhYw_4gQ
4_F5hIX6IUc9j1DNqvI2qg
o2jbfJzQ4Q7oavMUWuMEBg
